# Diabetes Dataset – Data Profiling & Cleaning

This notebook performs comprehensive data cleaning on the UCI diabetes dataset in preparation for modeling hospital readmission. 

**Goals:**
- Inspect and clean categorical, numeric, and diagnostic fields
- Normalize inconsistent formats (e.g., ICD-9 codes, weight ranges)
- Handle missing or unknown values
- Prepare a clean, model-ready dataset for downstream exploration and modeling

The graphical EDA and modeling steps are covered in separate notebooks.

## 1. Load Raw Data and Initial Schema Inspection
- Read CSV and check structure with `.info()`
- Identify column types and missing values

In [1]:
import pandas as pd
import numpy as np

raw_data = pd.read_csv('../data/diabetic_data.csv')

In [2]:
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    101766 non-null  object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                101766 non-null  object
 11  medical_specialty         101766 non-null  object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

## 2. Initial Exploration of Categorical (String) Columns
- Slice and inspect subsets of object-type fields
- Focus on early variables: race, gender, age, weight, payer_code, medical_specialty

In [3]:
string_cols = raw_data.select_dtypes('object').columns
string_cols.size

37

In [4]:
raw_data.loc[:, string_cols[0:13]].head()

,race,gender,age,weight,payer_code,medical_specialty,diag_1,diag_2,diag_3,max_glu_serum,A1Cresult,metformin,repaglinide
0,Caucasian,Female,[0-10),?,?,Pediatrics-Endocrinology,250.83,?,?,NaN,NaN,No,No
1,Caucasian,Female,[10-20),?,?,?,276,250.01,255,NaN,NaN,No,No
2,AfricanAmerican,Female,[20-30),?,?,?,648,250,V27,NaN,NaN,No,No
3,Caucasian,Male,[30-40),?,?,?,8,250.43,403,NaN,NaN,No,No
4,Caucasian,Male,[40-50),?,?,?,197,157,250,NaN,NaN,No,No


In [5]:
raw_data.loc[:, string_cols[14:27]].head()

,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide
0,No,No,No,No,No,No,No,No,No,No,No,No,No
1,No,No,No,No,No,No,No,No,No,No,No,No,No
2,No,No,No,Steady,No,No,No,No,No,No,No,No,No
3,No,No,No,No,No,No,No,No,No,No,No,No,No
4,No,No,No,Steady,No,No,No,No,No,No,No,No,No


In [6]:
raw_data.loc[:, string_cols[28:37]].head()

,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,No,No,No,No,No,No,No,No,NO
1,Up,No,No,No,No,No,Ch,Yes,>30
2,No,No,No,No,No,No,No,Yes,NO
3,Up,No,No,No,No,No,Ch,Yes,NO
4,Steady,No,No,No,No,No,Ch,Yes,NO


In [7]:
raw_data.loc[:, string_cols[0:13]].describe()

,race,gender,age,weight,payer_code,medical_specialty,diag_1,diag_2,diag_3,max_glu_serum,A1Cresult,metformin,repaglinide
count,101766,101766,101766,101766,101766,101766,101766,101766,101766,5346,17018,101766,101766
unique,6,3,10,10,18,73,717,749,790,3,3,4,4
top,Caucasian,Female,[70-80),?,?,?,428,276,250,Norm,>8,No,No
freq,76099,54708,26068,98569,40256,49949,6862,6752,11555,2597,8216,81778,100227


Lets work these left to right. Starting with race:

In [8]:
raw_data['race'].value_counts()

race
Caucasian          76099
AfricanAmerican    19210
?                   2273
Hispanic            2037
Other               1506
Asian                641
Name: count, dtype: int64

This is OK; we'll probably overwrite the ? with NaN.

In [9]:
raw_data['gender'].value_counts()

gender
Female             54708
Male               47055
Unknown/Invalid        3
Name: count, dtype: int64

Unknown/invalid can become NaN

In [10]:
raw_data['age'].value_counts()

age
[70-80)     26068
[60-70)     22483
[50-60)     17256
[80-90)     17197
[40-50)      9685
[30-40)      3775
[90-100)     2793
[20-30)      1657
[10-20)       691
[0-10)        161
Name: count, dtype: int64

Nothing missing; We may simplify this a little bit during cleanup.

In [11]:
raw_data['weight'].value_counts()

weight
?            98569
[75-100)      1336
[50-75)        897
[100-125)      625
[125-150)      145
[25-50)         97
[0-25)          48
[150-175)       35
[175-200)       11
>200             3
Name: count, dtype: int64

Again, nothing missing (as NaN); We may simplify this a little bit during cleanup.

In [12]:
raw_data['payer_code'].info()
sorted(raw_data['payer_code'].unique())

<class 'pandas.core.series.Series'>
RangeIndex: 101766 entries, 0 to 101765
Series name: payer_code
Non-Null Count   Dtype 
--------------   ----- 
101766 non-null  object
dtypes: object(1)
memory usage: 795.2+ KB


['?',
 'BC',
 'CH',
 'CM',
 'CP',
 'DM',
 'FR',
 'HM',
 'MC',
 'MD',
 'MP',
 'OG',
 'OT',
 'PO',
 'SI',
 'SP',
 'UN',
 'WC']

Nothing crazy; NA's will go into the '?'

In [13]:
raw_data['medical_specialty'].info()
sorted(raw_data['medical_specialty'].unique())

<class 'pandas.core.series.Series'>
RangeIndex: 101766 entries, 0 to 101765
Series name: medical_specialty
Non-Null Count   Dtype 
--------------   ----- 
101766 non-null  object
dtypes: object(1)
memory usage: 795.2+ KB


['?',
 'AllergyandImmunology',
 'Anesthesiology',
 'Anesthesiology-Pediatric',
 'Cardiology',
 'Cardiology-Pediatric',
 'DCPTEAM',
 'Dentistry',
 'Dermatology',
 'Emergency/Trauma',
 'Endocrinology',
 'Endocrinology-Metabolism',
 'Family/GeneralPractice',
 'Gastroenterology',
 'Gynecology',
 'Hematology',
 'Hematology/Oncology',
 'Hospitalist',
 'InfectiousDiseases',
 'InternalMedicine',
 'Nephrology',
 'Neurology',
 'Neurophysiology',
 'Obsterics&Gynecology-GynecologicOnco',
 'Obstetrics',
 'ObstetricsandGynecology',
 'Oncology',
 'Ophthalmology',
 'Orthopedics',
 'Orthopedics-Reconstructive',
 'Osteopath',
 'Otolaryngology',
 'OutreachServices',
 'Pathology',
 'Pediatrics',
 'Pediatrics-AllergyandImmunology',
 'Pediatrics-CriticalCare',
 'Pediatrics-EmergencyMedicine',
 'Pediatrics-Endocrinology',
 'Pediatrics-Hematology-Oncology',
 'Pediatrics-InfectiousDiseases',
 'Pediatrics-Neurology',
 'Pediatrics-Pulmonology',
 'Perinatology',
 'PhysicalMedicineandRehabilitation',
 'PhysicianNo

This one will take a little more manual tweaking. There's specialties and subspecialties (and sub-sub-specialties); Questions arise (is Cardiology-Pediatric meaningfully different than Pediatric-Cardiology?).

## 3. Diagnosis Code Analysis and Validation
- Explore diag_1, diag_2, diag_3 using regex
- Check format consistency (ICD-9 compliance)
- Identify malformed and abbreviated codes

Now let's look at our diag_N fields. From our variable dictionary we know that the diag_N are primary, secondary, and tertiary diagnoses from the ICD9 (International Classification of Diseases). We're told there are missing values; let's see how they might be coded. ICD9 codes are numeric, with some having 'E' and 'V' preceding. We can use a regex to toss anything that conforms to that, and then see what's left.

In [14]:
print(raw_data['diag_1'].str.contains(r'^\d{3}$|^[EV]\d{2}$', na=False).sum())
print(raw_data['diag_2'].str.contains(r'^\d{3}$|^[EV]\d{2}$', na=False).sum())
print(raw_data['diag_3'].str.contains(r'^\d{3}$|^[EV]\d{2}$', na=False).sum())

all_codes = pd.concat([raw_data['diag_1'], raw_data['diag_2'], raw_data['diag_3']], ignore_index=True)
print(all_codes.__len__())

90576
92394
91970
305298


In [15]:
## Now to see what non-conformers might be in here.
all_codes[~all_codes.str.contains(r'^\d{3}$|^[EV]\d{2}$')].value_counts().head(25)

250.02    4118
250.6     3158
250.01    2499
38        2325
250.8     2201
?         1802
250.7     1142
41        1114
250.13     967
8          904
250.4      866
250.11     722
250.82     702
70         600
250.03     568
250.12     552
250.41     518
250.92     402
250.1      350
250.81     330
250.42     323
E878       277
E849       273
E888       240
E885       229
Name: count, dtype: int64

So they aren't all exactly three numbers; that's OK. The xxx.nn formatting is more detail than the dataset promised to deliver. Let's see what we get for one- and two-digit numbers, ignoring V and E for now:

In [16]:
all_codes = pd.concat([raw_data['diag_1'], raw_data['diag_2'], raw_data['diag_3']], ignore_index=True)
all_codes[all_codes.str.contains(r'^\d{1,2}$', na=False)].value_counts()

38    2325
41    1114
8      904
70     600
79     161
42     153
53     112
9       58
94      54
54      36
47      27
34      23
11      15
3       15
35      15
78      14
5       13
40      12
88      12
66       8
82       7
31       7
49       6
27       6
75       5
7        5
52       3
58       2
23       2
36       2
57       2
48       2
39       2
97       1
98       1
84       1
61       1
10       1
46       1
96       1
99       1
17       1
14       1
Name: count, dtype: int64

It's reasonable to assume that one- and two-digit numbers without V or E codes should be 00N or 0NN, not NN; we'll note to make that change for the sake of completeness/coherence; depending on how we choose to handle these codes, it may not matter (if we convert them all to numerics eventually, for example - but we don't know exactly how we want to handle that at this stage). V codes are *only* two digits behind the V. Let's make sure that's the case:

In [17]:
all_e_codes = all_codes[all_codes.str.contains(r'^E\d+', na=False)]
all_v_codes = all_codes[all_codes.str.contains(r'^V\d+', na=False)]
all_e_codes.info()
print("\n")
all_v_codes.info()

<class 'pandas.core.series.Series'>
Index: 1976 entries, 73324 to 305239
Series name: None
Non-Null Count  Dtype 
--------------  ----- 
1976 non-null   object
dtypes: object(1)
memory usage: 30.9+ KB


<class 'pandas.core.series.Series'>
Index: 7263 entries, 23 to 305215
Series name: None
Non-Null Count  Dtype 
--------------  ----- 
7263 non-null   object
dtypes: object(1)
memory usage: 113.5+ KB


In [18]:
print(all_e_codes.unique())

['E909' 'E878' 'E812' 'E932' 'E888' 'E939' 'E937' 'E944' 'E870' 'E849'
 'E950' 'E934' 'E935' 'E915' 'E885' 'E880' 'E879' 'E890' 'E817' 'E931'
 'E924' 'E942' 'E947' 'E930' 'E858' 'E929' 'E933' 'E900' 'E936' 'E941'
 'E884' 'E928' 'E965' 'E813' 'E814' 'E927' 'E905' 'E917' 'E868' 'E854'
 'E918' 'E850' 'E887' 'E881' 'E829' 'E919' 'E916' 'E819' 'E826' 'E938'
 'E816' 'E906' 'E818' 'E980' 'E853' 'E968' 'E882' 'E821' 'E945' 'E883'
 'E949' 'E920' 'E956' 'E904' 'E943' 'E861' 'E852' 'E876' 'E855' 'E815'
 'E822' 'E894' 'E828' 'E865' 'E946' 'E966' 'E922' 'E901' 'E892' 'E886'
 'E987' 'E912' 'E955' 'E864' 'E825']


Good! Let's look at the V codes now. These should all be three digits (leading zero), and might have a period with additional decimals behind it.

In [19]:
print(all_v_codes.unique())
# Did not end up needing, but another way to do it:
# malformed_v = all_codes[all_codes.str.contains(r'^V\d{1}$|^V\d{3,}$', na=False)]
# malformed_v.info()

['V57' 'V58' 'V55' 'V53' 'V45' 'V66' 'V56' 'V26' 'V71' 'V54' 'V67' 'V60'
 'V43' 'V63' 'V25' 'V70' 'V07' 'V51' 'V15' 'V10' 'V42' 'V44' 'V65' 'V12'
 'V23' 'V17' 'V72' 'V49' 'V18' 'V14' 'V46' 'V64' 'V61' 'V08' 'V62' 'V09'
 'V11' 'V16' 'V13' 'V85' 'V02' 'V50' 'V03' 'V69' 'V86' 'V27' 'V22' 'V01'
 'V06']


Also good. Moving on.

## 4. Inspection of Glucose and A1C Test Fields
- Analyze levels and distribution for `max_glu_serum` and `A1Cresult`

In [20]:
print(raw_data['max_glu_serum'].dropna().head())
raw_data['max_glu_serum'].value_counts()

133    >300
135    >300
142    Norm
144    Norm
153    Norm
Name: max_glu_serum, dtype: object


max_glu_serum
Norm    2597
>200    1485
>300    1264
Name: count, dtype: int64

In [21]:
print(raw_data['A1Cresult'].dropna().head())
raw_data['A1Cresult'].value_counts()

26       >7
74       >7
98       >8
103    Norm
107    Norm
Name: A1Cresult, dtype: object


A1Cresult
>8      8216
Norm    4990
>7      3812
Name: count, dtype: int64

So for these two, we have a lot of NA's, then three levels (normal, high, real high). The NA's (we're told) indicates that the testing was not performed (possibly a significant item for our model?).

The next round of string columns cover a whole lot of drugs & drug combinations.

## 5. Drug Treatment Columns Summary
- Validate consistency across 23 drug-related columns
- Confirm value ranges (No, Steady, Up, Down)

In [22]:
drug_cols = raw_data[['metformin','repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
                      'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone',
                      'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide',
                      'examide', 'citoglipton', 'insulin', 'glyburide-metformin', 'glipizide-metformin',
                      'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone']]
print(drug_cols.isna().sum())
drug_cols.stack().value_counts()

metformin                   0
repaglinide                 0
nateglinide                 0
chlorpropamide              0
glimepiride                 0
acetohexamide               0
glipizide                   0
glyburide                   0
tolbutamide                 0
pioglitazone                0
rosiglitazone               0
acarbose                    0
miglitol                    0
troglitazone                0
tolazamide                  0
examide                     0
citoglipton                 0
insulin                     0
glyburide-metformin         0
glipizide-metformin         0
glimepiride-pioglitazone    0
metformin-rosiglitazone     0
metformin-pioglitazone      0
dtype: int64


No        2220564
Steady      90802
Up          14865
Down        14387
Name: count, dtype: int64

Excellent - all values present and none malformed to have to clean. All drug columns are "No", "Steady", "Up", or "Down".

## 6. Key Binary Flags: Change, DiabetesMed, Readmitted
- Map values to logical/boolean or ordinal encodings

In [23]:
print(raw_data['change'].isna().sum())
print(raw_data['change'].value_counts())
print("\n")
print(raw_data['diabetesMed'].isna().sum())
print(raw_data['diabetesMed'].value_counts())

0
change
No    54755
Ch    47011
Name: count, dtype: int64


0
diabetesMed
Yes    78363
No     23403
Name: count, dtype: int64


These look good to go; onto the last, and possibly most important string column - readmitted is what we're trying to build a model to predict. 

In [24]:
print(raw_data['readmitted'].isna().sum())
print(raw_data['readmitted'].value_counts())

0
readmitted
NO     54864
>30    35545
<30    11357
Name: count, dtype: int64


We will map these to numeric values.

## 7. Categorical Cleanup and Normalization
- Replace `?` and invalid values with `NaN`
- Convert age and weight from ranges to numeric
- Map binary and ordinal strings to integers/booleans

In [25]:
## Fresh new dataframe, leaving raw_data unmolested
clean_data = raw_data.copy() 

# Direct replacement of just those values in our string columns.
clean_data[string_cols] = clean_data[string_cols].replace('?', np.nan) 

# Replacement of "Unknown/Invalid" in gender to be NA
clean_data['gender'] = clean_data['gender'].replace('Unknown/Invalid', np.nan) 

# one-shot regex replacement with the max value for the bucket
clean_data['age'] = clean_data['age'].str.replace(r'\[(\d+)-(\d+)\)', r'\2', regex=True).astype(int) 

# For weight, we want to keep the '>200', so its slightly different regex than for age.
clean_data['weight'] = clean_data['weight'].str.replace(r'^\[(\d+)-(\d+)\)', r'\2', regex=True)
# Remap these strings to bools
clean_data['change'] = clean_data['change'].map({'No': False, 'Ch': True}) 
clean_data['diabetesMed'] = clean_data['diabetesMed'].map({'No': False, 'Yes': True})

# Remap readmitted to numerics. 
clean_data['readmitted'] = clean_data['readmitted'].map({'NO': -1, '>30': 0, '<30': 1})

## 8. Diagnosis Code Normalization (Final Pass)
- Pad short numeric codes
- Remove periods
- Validate post-cleaning format

In [26]:
diag_cols = ['diag_1', 'diag_2', 'diag_3']

# Append leading zeros to the single- and double-digit codes
clean_data[diag_cols] = clean_data[diag_cols].apply(lambda x: x.str.replace(r'^(\d)$', r'00\1', regex=True).str.replace(r'^(\d{2})$', r'0\1', regex=True))

# Verify the change - no more single- or double-digit strings.
clean_data[diag_cols].apply(lambda x: x.str.contains(r'^\d{1,2}$', na=False)).sum()

diag_1    0
diag_2    0
diag_3    0
dtype: int64

We'll use an ICD9 map (https://github.com/drobbins/ICD9), which expects anywhere from a three to five digit number to represent the hierarchical diagnosis codes. We can delete the periods in our strings, which will yield three- to five-digit final values.

In [27]:
for i, col in enumerate(diag_cols):
    clean_data[col] = clean_data[col].str.replace('.', '')
    
# Check that we have between three and five digit codes excluding V&E codes
for col in diag_cols:
    # Items that don't match: not 3-5 digits OR start with V/E
    non_matching = clean_data[col][
        clean_data[col].str.contains(r'^\d', na=False) &  # starts with digit
        ~clean_data[col].str.contains(r'^\d{3,5}$', na=False)  # but not 3-5 digits
    ]
    print(f"{col}: {non_matching.unique()}")

diag_1: []
diag_2: []
diag_3: []


Now we are in good shape to map our values to the ICD 9 map we got from the open source.

## 9. Medical Specialty Cleaning and Hierarchy Extraction
- Normalize specialties and subspecialties
- Handle malformed or unknown labels
- Create new column: `medical_subspecialty`

Now to handle the medical specialty column. This is going to require some framing assumptions that will need to be checked, much later. While the source of the data says there are 84 distinct values, we show 73 including NA (unknown):

In [28]:
clean_data['medical_specialty'].unique().__len__()

73

In [29]:
sorted(clean_data['medical_specialty'].astype(str).unique())

['AllergyandImmunology',
 'Anesthesiology',
 'Anesthesiology-Pediatric',
 'Cardiology',
 'Cardiology-Pediatric',
 'DCPTEAM',
 'Dentistry',
 'Dermatology',
 'Emergency/Trauma',
 'Endocrinology',
 'Endocrinology-Metabolism',
 'Family/GeneralPractice',
 'Gastroenterology',
 'Gynecology',
 'Hematology',
 'Hematology/Oncology',
 'Hospitalist',
 'InfectiousDiseases',
 'InternalMedicine',
 'Nephrology',
 'Neurology',
 'Neurophysiology',
 'Obsterics&Gynecology-GynecologicOnco',
 'Obstetrics',
 'ObstetricsandGynecology',
 'Oncology',
 'Ophthalmology',
 'Orthopedics',
 'Orthopedics-Reconstructive',
 'Osteopath',
 'Otolaryngology',
 'OutreachServices',
 'Pathology',
 'Pediatrics',
 'Pediatrics-AllergyandImmunology',
 'Pediatrics-CriticalCare',
 'Pediatrics-EmergencyMedicine',
 'Pediatrics-Endocrinology',
 'Pediatrics-Hematology-Oncology',
 'Pediatrics-InfectiousDiseases',
 'Pediatrics-Neurology',
 'Pediatrics-Pulmonology',
 'Perinatology',
 'PhysicalMedicineandRehabilitation',
 'PhysicianNotFound

Let's go ahead and:

- convert 'Surgeon' to 'Surgery'
- convert 'SurgicalSpecialty' to 'Surgery' primary and 'Other' secondary ('Surgery-OtherSpecialty' before we split)
- create a new column for subspecialty
    - splitting on the dash
    - filling with NA's for those without subspecialties
- convert "Obsterics&Gynecology-GynecologicOnco"'s primary specialty to "ObstetricsandGynecology" to conform
- convert 'PhysicianNotFound' to NA (assumption being made here...)



In [30]:
clean_data['medical_specialty'] = clean_data['medical_specialty'].str.replace('Surgeon', 'Surgery')
clean_data['medical_specialty'] = clean_data['medical_specialty'].str.replace('SurgicalSpecialty', 'Surgery-OtherSpecialty')

clean_data['medical_subspecialty'] = clean_data['medical_specialty'].str.split('-', n=1, expand=True)[1]
clean_data['medical_specialty'] = clean_data['medical_specialty'].str.split('-', n=1, expand=True)[0]

clean_data['medical_specialty'] = clean_data['medical_specialty'].str.replace('Obsterics&Gynecology', 'ObstetricsandGynecology')
clean_data.loc[clean_data['medical_specialty'].str.contains('PhysicianNotFound', na = False), 'medical_specialty'] = np.nan

Let's see how these did:

In [31]:
sorted(clean_data['medical_specialty'].astype(str).unique()) # Sorted doesn't like booleans (NA)

['AllergyandImmunology',
 'Anesthesiology',
 'Cardiology',
 'DCPTEAM',
 'Dentistry',
 'Dermatology',
 'Emergency/Trauma',
 'Endocrinology',
 'Family/GeneralPractice',
 'Gastroenterology',
 'Gynecology',
 'Hematology',
 'Hematology/Oncology',
 'Hospitalist',
 'InfectiousDiseases',
 'InternalMedicine',
 'Nephrology',
 'Neurology',
 'Neurophysiology',
 'Obstetrics',
 'ObstetricsandGynecology',
 'Oncology',
 'Ophthalmology',
 'Orthopedics',
 'Osteopath',
 'Otolaryngology',
 'OutreachServices',
 'Pathology',
 'Pediatrics',
 'Perinatology',
 'PhysicalMedicineandRehabilitation',
 'Podiatry',
 'Proctology',
 'Psychiatry',
 'Psychology',
 'Pulmonology',
 'Radiologist',
 'Radiology',
 'Resident',
 'Rheumatology',
 'Speech',
 'SportsMedicine',
 'Surgery',
 'Urology',
 'nan']

In [32]:
sorted(clean_data['medical_subspecialty'].astype(str).unique()) # Sorted doesn't like booleans (NA)

['Addictive',
 'AllergyandImmunology',
 'Cardiovascular',
 'Cardiovascular/Thoracic',
 'Child/Adolescent',
 'Colon&Rectal',
 'CriticalCare',
 'EmergencyMedicine',
 'Endocrinology',
 'General',
 'GynecologicOnco',
 'Hematology-Oncology',
 'InfectiousDiseases',
 'Maxillofacial',
 'Metabolism',
 'Neuro',
 'Neurology',
 'None',
 'OtherSpecialty',
 'Pediatric',
 'Plastic',
 'PlasticwithinHeadandNeck',
 'Pulmonology',
 'Reconstructive',
 'Thoracic',
 'Vascular',
 'nan']

Good enough for now - maybe we come back and clean this up more later. We need to move on to making sure all our numeric data is in good shape to support actual exploration of the dataset.

## 10. Numeric Columns Validation
- Confirm integer types and value ranges
- Check for malformed or unexpected entries

In [33]:
num_cols = raw_data.select_dtypes(exclude='object').columns
raw_data[num_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 13 columns):
 #   Column                    Non-Null Count   Dtype
---  ------                    --------------   -----
 0   encounter_id              101766 non-null  int64
 1   patient_nbr               101766 non-null  int64
 2   admission_type_id         101766 non-null  int64
 3   discharge_disposition_id  101766 non-null  int64
 4   admission_source_id       101766 non-null  int64
 5   time_in_hospital          101766 non-null  int64
 6   num_lab_procedures        101766 non-null  int64
 7   num_procedures            101766 non-null  int64
 8   num_medications           101766 non-null  int64
 9   number_outpatient         101766 non-null  int64
 10  number_emergency          101766 non-null  int64
 11  number_inpatient          101766 non-null  int64
 12  number_diagnoses          101766 non-null  int64
dtypes: int64(13)
memory usage: 10.1 MB


All numerics look to be in good shape. We're told in the data description from the source that these do not contain missing values. As encounter_id and patient_nbr are unique identifiers, but the other values aren't, let's see if those others have any unexpected or malformed entries:

In [34]:
# Drop encounter_id and patient_nbr from num_cols
num_cols = num_cols.drop(['encounter_id', 'patient_nbr'])

# pd.concat([df[col] for col in df.columns])
all_nums = raw_data[num_cols].stack()

In [35]:
print(', '.join(sorted(all_nums.astype(str).unique())))

0, 1, 10, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 11, 111, 113, 114, 118, 12, 120, 121, 126, 129, 13, 132, 14, 15, 16, 17, 18, 19, 2, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 3, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 4, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 5, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 6, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 7, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 8, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 9, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99


So, nothing wild or crazy; good! We can close out our pre-exploration data preparation at this point and move on to some early exploratory analysis.

## 11. Save Cleaned Dataset
- Export cleaned data to `diabetes_clean.pkl` for future analysis

In [36]:
clean_data.to_pickle("../data/diabetes_clean.pkl")